In [4]:
!pip install onnxruntime-gpu
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 111.4 MB/s eta 0:00:00a 0:00:01


In [17]:
import onnxruntime as ort
import cv2
import numpy as np
import time

In [18]:
available_providers = ort.get_available_providers()
print(available_providers)

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [19]:
# model = ort.InferenceSession('model/end2end.onnx', providers=["CUDAExecutionProvider"])
model = ort.InferenceSession('model/Posenet-Mobilenet.onnx', providers=["CUDAExecutionProvider"])

In [20]:
input_names = [input.name for input in model.get_inputs()]
output_names = [output.name for output in model.get_outputs()]

input_details = model.get_inputs()
print(input_details[0])

NodeArg(name='image', type='tensor(float)', shape=[1, 3, 513, 257])


In [21]:
def mmpose_data_preprocessor(image: np.ndarray) -> np.ndarray:
    image = cv2.resize(image, (257, 513))
    mean = np.array([123.675, 116.28, 103.53], dtype=np.float32)
    std = np.array([58.395, 57.12, 57.375], dtype=np.float32)

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # rgb_image = rgb_image.astype(np.float32)

    normalized_image = (rgb_image - mean.reshape(1, 1, 3)) / std.reshape(1, 1, 3)

    chw_image = normalized_image.transpose((2, 0, 1))

    return chw_image

In [34]:
import cv2

cap = cv2.VideoCapture('dance.mkv') # Replace with your video file path

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret: # Break the loop if no more frames are available
        break
    # print(frame.shape)
    pp_img = mmpose_data_preprocessor(frame)
    pp_img = np.expand_dims(pp_img, axis=0)
    # print(pp_img.shape)
    start = time.perf_counter()
    output = model.run(output_names, {input_names[0]: pp_img})
    end = time.perf_counter()
    if frame_count == 6*30:
        break
    # print(len(output))
    # for o in output:
    #     print(o.shape)
    # print(end-start)
    


    frame_count += 1

print(frame_count)
cap.release()

180


In [15]:
import posenet

In [ ]:
parser.add_argument('--model', type=int, default=101)
parser.add_argument('--cam_id', type=int, default=0)
parser.add_argument('--cam_width', type=int, default=1280)
parser.add_argument('--cam_height', type=int, default=720)
parser.add_argument('--scale_factor', type=float, default=0.7125)


while True:
    input_image, display_image, output_scale = posenet.read_cap(
        cap, scale_factor=0.7125, output_stride=output_stride)

    with torch.no_grad():
        input_image = torch.Tensor(input_image).cuda()

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = model(input_image)

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(0),
            offsets_result.squeeze(0),
            displacement_fwd_result.squeeze(0),
            displacement_bwd_result.squeeze(0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.15)

    keypoint_coords *= output_scale


In [35]:
import numpy as np

# Standard PoseNet keypoint mapping (by index)
KEYPOINT_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar",
    "leftShoulder", "rightShoulder", "leftElbow", "rightElbow",
    "leftWrist", "rightWrist", "leftHip", "rightHip",
    "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]

def decode_single_person_pose(
    heatmaps_tensor: np.ndarray,  # Shape: (1, 17, H, W)
    offsets_tensor: np.ndarray,   # Shape: (1, 34, H, W)
    output_stride: int,
    original_image_shape: tuple,  # (original_height, original_width)
    model_input_size: tuple,      # (height, width) image was resized to for model input
    confidence_threshold: float = 0.2
) -> dict:
    """
    Decodes keypoint coordinates for a single person from raw PoseNet heatmaps
    and offset tensors, scaling them to the original image dimensions.

    This function is primarily for single-person scenarios or to get the
    strongest keypoint detections overall. It does NOT perform multi-person
    decoding (i.e., it won't group keypoints for multiple individuals).

    Args:
        heatmaps_tensor (np.ndarray): The heatmap tensor (1, 17, H, W) in NumPy format.
        offsets_tensor (np.ndarray): The offset tensor (1, 34, H, W) in NumPy format.
        output_stride (int): The stride of the PoseNet model (e.g., 16 or 32).
        original_image_shape (tuple): (height, width) of the image before any resizing.
        model_input_size (tuple): (height, width) of the image *after* resizing for the model input.
                                 e.g., (257, 257) or (513, 513).
        confidence_threshold (float): Minimum confidence for a keypoint to be returned.

    Returns:
        dict: A dictionary where keys are keypoint names (e.g., 'nose', 'leftWrist')
              and values are tuples (x, y, confidence) in original image coordinates.
              Keypoints not meeting the confidence threshold will be `None`.
    """
    if heatmaps_tensor.ndim != 4 or offsets_tensor.ndim != 4:
        raise ValueError("Input tensors must be 4-dimensional (batch, channels, H, W).")
    if heatmaps_tensor.shape[0] != 1 or offsets_tensor.shape[0] != 1:
        raise ValueError("This function expects a batch size of 1 for simplicity.")
    if heatmaps_tensor.shape[1] != len(KEYPOINT_NAMES): # Channels dimension
        raise ValueError(f"Heatmap channels ({heatmaps_tensor.shape[1]}) must match number of keypoints ({len(KEYPOINT_NAMES)}).")
    if offsets_tensor.shape[1] != len(KEYPOINT_NAMES) * 2: # Channels dimension
        raise ValueError(f"Offset channels ({offsets_tensor.shape[1]}) must match 2 * number of keypoints ({len(KEYPOINT_NAMES)*2}).")

    # Extract relevant parts, removing batch dimension
    # Now, channels come first: (17, H, W) for heatmaps, (34, H, W) for offsets
    heatmaps = heatmaps_tensor.squeeze(0)
    offsets = offsets_tensor.squeeze(0)

    # Get spatial dimensions from heatmaps (channels, H, W)
    num_keypoints_hm, heatmap_height, heatmap_width = heatmaps.shape

    detected_keypoints = {}
    
    # Calculate scaling factors to map from model input coordinates to original image coordinates
    original_h, original_w = original_image_shape
    model_input_h, model_input_w = model_input_size
    
    scale_x = original_w / model_input_w
    scale_y = original_h / model_input_h

    for kp_idx, kp_name in enumerate(KEYPOINT_NAMES):
        # Select the heatmap for the current keypoint: (H, W)
        keypoint_heatmap = heatmaps[kp_idx, :, :]

        # Find the max confidence score and its location (y, x) in the heatmap
        max_score = np.max(keypoint_heatmap)

        if max_score < confidence_threshold:
            # Keypoint not confident enough
            detected_keypoints[kp_name] = None
            continue

        # Get the (y, x) indices of the maximum score
        max_score_indices = np.unravel_index(np.argmax(keypoint_heatmap), keypoint_heatmap.shape)
        coarse_y, coarse_x = max_score_indices

        # Retrieve the corresponding offsets
        # Offsets channels are typically ordered as [x0, y0, x1, y1, ...] for each keypoint
        # So, for kp_idx, x_offset_channel = kp_idx * 2, y_offset_channel = kp_idx * 2 + 1
        offset_x = offsets[kp_idx * 2, coarse_y, coarse_x]
        offset_y = offsets[kp_idx * 2 + 1, coarse_y, coarse_x]

        # Calculate raw (unscaled) keypoint coordinates in the model's input coordinate system
        # These are usually float values
        raw_x = coarse_x * output_stride + offset_x
        raw_y = coarse_y * output_stride + offset_y

        # Scale to original image dimensions and convert to integer pixels
        final_x = int(round(raw_x * scale_x))
        final_y = int(round(raw_y * scale_y))

        # Clamp coordinates to be within original image bounds (0 to width-1, 0 to height-1)
        final_x = max(0, min(final_x, original_w - 1))
        final_y = max(0, min(final_y, original_h - 1))

        detected_keypoints[kp_name] = (final_x, final_y, max_score)

    return detected_keypoints

# --- Example Usage (Simulating Output Tensors) ---
if __name__ == "__main__":
    # Define example parameters (you'll need to know these from your model's pre-processing)
    ORIGINAL_IMAGE_SHAPE = (1080, 1920) # (height, width) of your actual input image
    MODEL_INPUT_SIZE = (257, 513)      # (height, width) the image was resized/padded to for model input
                                       # Derived from 33 * 16 and 17 * 16 if output_stride = 16
    OUTPUT_STRIDE = 16                 # Common for MobileNet-PoseNet

    # Calculate heatmap/offset map dimensions based on model input size and output stride
    # H = model_input_height / output_stride, W = model_input_width / output_stride
    HM_H = MODEL_INPUT_SIZE[0] // OUTPUT_STRIDE # 528 // 16 = 33
    HM_W = MODEL_INPUT_SIZE[1] // OUTPUT_STRIDE # 272 // 16 = 17


    # --- Run the decoding function ---
    keypoints_on_original_image = decode_single_person_pose(
        heatmaps_tensor=output[0],
        offsets_tensor=output[1],
        output_stride=OUTPUT_STRIDE,
        original_image_shape=ORIGINAL_IMAGE_SHAPE,
        model_input_size=MODEL_INPUT_SIZE,
        confidence_threshold=0.00001 # Default threshold
    )

    print("\n--- Detected Keypoints on Original Image (Confidence >= 0.2) ---")
    for kp_name, coords in keypoints_on_original_image.items():
        if coords:
            x, y, confidence = coords
            print(f"{kp_name:15}: X={x:<5}, Y={y:<5}, Confidence={confidence:.4f}")
        else:
            print(f"{kp_name:15}: Not detected (or below threshold)")

    # --- Test with a higher confidence threshold ---
    print("\n--- Detected Keypoints on Original Image (Confidence >= 0.9) ---")
    keypoints_high_confidence = decode_single_person_pose(
        heatmaps_tensor=output[0],
        offsets_tensor=output[1],
        output_stride=OUTPUT_STRIDE,
        original_image_shape=ORIGINAL_IMAGE_SHAPE,
        model_input_size=MODEL_INPUT_SIZE,
        confidence_threshold=0.9
    )
    for kp_name, coords in keypoints_high_confidence.items():
        if coords:
            x, y, confidence = coords
            print(f"{kp_name:15}: X={x:<5}, Y={y:<5}, Confidence={confidence:.4f}")
        else:
            print(f"{kp_name:15}: Not detected (or below threshold)")


--- Detected Keypoints on Original Image (Confidence >= 0.2) ---
nose           : X=527  , Y=765  , Confidence=0.4783
leftEye        : X=502  , Y=786  , Confidence=0.3494
rightEye       : X=541  , Y=825  , Confidence=0.3767
leftEar        : X=696  , Y=818  , Confidence=0.2714
rightEar       : X=464  , Y=782  , Confidence=0.4402
leftShoulder   : X=579  , Y=982  , Confidence=0.7767
rightShoulder  : X=379  , Y=1038 , Confidence=0.3226
leftElbow      : X=633  , Y=1079 , Confidence=0.3119
rightElbow     : X=476  , Y=1079 , Confidence=0.1440
leftWrist      : X=671  , Y=1079 , Confidence=0.3802
rightWrist     : X=749  , Y=1079 , Confidence=0.1498
leftHip        : X=892  , Y=1079 , Confidence=0.0937
rightHip       : X=661  , Y=1079 , Confidence=0.2133
leftKnee       : X=721  , Y=1079 , Confidence=0.0655
rightKnee      : X=729  , Y=1079 , Confidence=0.1018
leftAnkle      : X=956  , Y=1079 , Confidence=0.0111
rightAnkle     : X=297  , Y=1079 , Confidence=0.0148

--- Detected Keypoints on Origin

In [26]:
output[1].shape

(1, 34, 33, 17)